Aisha Muhammad Nawaz  
Pyspark Clustering Lab 5 L20-0921  
14th March 2024  

In [ ]:
# #Running on Colab
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=7f0b644ccf1252aee5105123987315475e688970075d84542e97342ac72f0acd
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [ ]:
#  Import the libraries we will need
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

# Create Spark session and ContextRun PySpark.
# create the session
conf = SparkConf().set("spark.ui.port","4050")
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.appName("DataFrame").config('spark.ui.port', '4050').getOrCreate()
spark

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-39a620b45ddf>:11 

In [ ]:
# sampletTest.txt has content as follows:
# 1.0 2.0 3.0
# 1.5 2.5 3.5
# 5.0 6.0 7.0
# 6.0 7.0 8.0
# 1.8 2.8 3.8

myData=spark.read.text('sampleTest.txt').rdd.map(lambda r: r[0])
myData.collect()

['0.0 0.0 0.0',
 '0.1 0.1 0.1',
 '0.2 0.2 0.2',
 '9.0 9.0 9.0',
 '9.1 9.1 9.1',
 '9.2 9.2 9.2']

In [ ]:
import numpy as np
def parseVector(line):
  return np.array([float(x) for x in line.split(' ')])
parsedData=myData.map(lambda x: parseVector(x))
parsedData.collect()

[array([0., 0., 0.]),
 array([0.1, 0.1, 0.1]),
 array([0.2, 0.2, 0.2]),
 array([9., 9., 9.]),
 array([9.1, 9.1, 9.1]),
 array([9.2, 9.2, 9.2])]

In [ ]:
# THIS IMPLEMENTATION MIGHT BE WRONG
K=2
convergedDistance=0.2
kPoints=parsedData.takeSample(withReplacement=False , num=K ,seed=1)
tempD=1.0
def findNearestCluster(x,kPoints):
  bestCluster=0
  minDist=float('inf')
  for i in range(len(kPoints)):
    dist=np.sum((x-kPoints[i])**2)
    if(dist<minDist):
      bestCluster=i
      minDist=dist
  return bestCluster

while tempD>convergedDistance:
  closestFound=parsedData.map(lambda x: (findNearestCluster(x,kPoints),(x,1))) #Assign Nearest Cluster to each Point
  closestFound2=closestFound.reduceByKey(lambda p1,p2: (p1[0]+p2[0],p1[1]+p2[1])) # Sum points and their count in each cluster
  newPoints=closestFound2.map(lambda point: (point[0],point[1][0]/point[1][1])).collect() #Find avg of points in each cluster
  # Note above the key is cluster
  # Now finding distance between new cluster centeres and old ones
  tempD=0.0
  sum=0
  for (ik,p) in newPoints:
    sum=sum+np.sum((kPoints[ik]-p)**2)
  tempD=sum
  # For structing purposes, assigning new clusters their values
  for (ik,p) in newPoints:
    kPoints[ik]=p

print('Final Centres ',kPoints)




Final Centres  [array([0.2, 0.2, 0.2]), array([13.8, 13.8, 13.8])]


In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator


# Read data from file
data = spark.read.text("sampleTest.txt").rdd.map(lambda x: x[0])

# Save the data in LibSVM format
data.saveAsTextFile("path_to_save_data_in_libsvm_format")

# Load the data in LibSVM format
dataset = spark.read.format("libsvm").load("path_to_save_data_in_libsvm_format")


# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Py4JJavaError: An error occurred while calling o1658.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 104.0 failed 1 times, most recent failure: Lost task 0.0 in stage 104.0 (TID 104) (1a6ae519ca96 executor driver): java.lang.NumberFormatException: For input string: "0.1"
	at java.lang.NumberFormatException.forInputString(NumberFormatException.java:65)
	at java.lang.Integer.parseInt(Integer.java:580)
	at java.lang.Integer.parseInt(Integer.java:615)
	at scala.collection.immutable.StringLike.toInt(StringLike.scala:310)
	at scala.collection.immutable.StringLike.toInt$(StringLike.scala:310)
	at scala.collection.immutable.StringOps.toInt(StringOps.scala:33)
	at org.apache.spark.mllib.util.MLUtils$.$anonfun$parseLibSVMRecord$2(MLUtils.scala:134)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(

In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
import numpy as np

# Define the MRJob class for KMeans algorithm
class MRKMeans(MRJob):

    def configure_args(self):
        super(MRKMeans, self).configure_args()
        self.add_passthru_arg('--k', default=3, type=int, help='Number of clusters (default: 3)')

    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init, mapper=self.mapper, reducer=self.reducer)
        ]

    # Initialize centroids with random data points
    def mapper_init(self):
        self.centroids = np.random.rand(self.options.k, 2)

    # Map each data point to its nearest centroid
    def mapper(self, _, line):
        data_point = np.array(list(map(float, line.split(','))))
        distances = np.linalg.norm(data_point - self.centroids, axis=1)
        nearest_centroid_index = np.argmin(distances)
        yield nearest_centroid_index, (data_point.tolist(), 1)

    # Reduce step to calculate new centroids
    def reducer(self, cluster_idx, data_points):
        total_points = 0
        new_centroid = np.zeros(2)
        for point, count in data_points:
            total_points += count
            new_centroid += np.array(point)
        new_centroid /= total_points
        yield cluster_idx, new_centroid.tolist()

if __name__ == '__main__':
    MRKMeans.run()


ModuleNotFoundError: No module named 'mrjob'